In [1]:
import os
import argparse
import pandas as pd
import numpy as np
import pickle
from utils_test import *

In [2]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [3]:
from keras.models import load_model

Using Theano backend.


In [19]:
name = 'helpdesk'
args = {
    'inputdir': '../input/{}/'.format(name),   
    'outputdir': './output_files/{}/'.format(name),
    'modelname': 'model_20-1.50.h5',
}

args = argparse.Namespace(**args)

In [20]:
test = pd.read_csv(args.inputdir+'test.csv')
test = transformDf(test)

In [21]:
with open(args.inputdir+'parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [22]:
with open(args.inputdir+'preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Utils

In [23]:
model = load_model(args.outputdir+args.modelname)

In [24]:
test_groupByCase = test.groupby(['CaseID'])

#get features all data
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

#get output all data
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

# Check longest case --> suffix: 2 to 7
test_len = findLongestLength(test_groupByCase)
test_len - 5

8

In [25]:
def getSuffix(suffix, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t):
    sentences_2 = []
    sentences_t_2 = []
    sentences_t2_2 = []
    sentences_t3_2 = []
    sentences_t4_2 = []
    next_chars_2 = []
    next_chars_t_2 = []

    for i in range(len(sentences)):
        if len(sentences[i]) == suffix and next_chars[i] != 'EOS':
            sentences_2.append(sentences[i])
            sentences_t_2.append(sentences_t[i])
            sentences_t2_2.append(sentences_t2[i])
            sentences_t3_2.append(sentences_t3[i])
            sentences_t4_2.append(sentences_t4[i])
            next_chars_2.append(next_chars[i])
            next_chars_t_2.append(next_chars_t[i])
    return sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2

In [26]:
def evalAct(true_label, pred_prob, targetchartoindice):
    #for log-loss: get probabilities
    gt_a = one_hot_encode(true_label, targetchartoindice)
    log_loss = metrics.log_loss(gt_a, pred_prob)
    print('Log-loss: {}'.format(log_loss))
    
    #for accuracy: get labels
    gt_a_label = true_label
    pred_a_label = getLabel(pred_prob, targetchartoindice)
    acc = metrics.accuracy_score(gt_a_label, pred_a_label)
    print('Accuracy: {}%'.format(acc*100))
    acc_3 = get_top3_accuracy(pred_prob, gt_a_label, targetchartoindice)
    print('Top 3 accuracy: {}%'.format(acc_3*100))

In [27]:
def evalTime(true_time, pred_time, divisor):
    #get time
    gt_t = true_time
    pred_t = inverseTime(pred_time, divisor)
    
    mse = metrics.mean_squared_error(gt_t, pred_t)
    mae = metrics.mean_absolute_error(gt_t, pred_t)
    median = metrics.median_absolute_error(gt_t, pred_t)
    
    print('Mean Squared Error: {0}s\t| {1} days'.format(mse, mse/86400))
    print('Mean Absolute Error: {0}s\t| {1} days'.format(mae, mae/86400))
    print('Median Absolute Error: {0}s\t| {1} days'.format(median, median/86400))

In [28]:
# DL distance
#distance.nlevenshtein(gt_a_label, pred_a_label) #0.1731066460587326

#damerau_levenshtein_distance(gt_a_label, pred_a_label) #784

#distance.jaccard(gt_a_label, pred_a_label) #0.4444444444444444

# Evaluation

In [29]:
max_suffix = 11

In [30]:
for i in range(2, max_suffix+1):
    print('----------Suffix {}------------'.format(i))
    sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 = getSuffix(i, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)
    X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)
    pred = model.predict(X_test, verbose=0)
    pred_a = pred[0]
    pred_t = pred[1]
    print('Evaluation of Activity')
    evalAct(next_chars_2, pred[0], targetchartoindice)
    print('Evaluation of Time')
    evalTime(next_chars_t_2, pred_t, divisor)
    print('\n')
    #Add gt and pred of each suffix
    if i == 2:
        full_gt_a = next_chars_2
        full_pred_a = pred_a
        full_gt_t = next_chars_t_2
        full_pred_t = pred_t
    else:
        full_gt_a += next_chars_2
        full_pred_a = np.concatenate((full_pred_a, pred_a), axis=0)
        full_gt_t += next_chars_t_2
        full_pred_t = np.concatenate((full_pred_t, pred_t), axis=0)

----------Suffix 2------------
Evaluation of Activity
Log-loss: 0.6852780553760628
Accuracy: 76.66666666666667%
Top 3 accuracy: 98.66666666666667%
Evaluation of Time
Mean Squared Error: 364379979318.4113s	| 4217360.871740872 days
Mean Absolute Error: 291981.7839178467s	| 3.3794187953454475 days
Median Absolute Error: 40914.55859375s	| 0.4735481318721065 days


----------Suffix 3------------
Evaluation of Activity
Log-loss: 1.1501585874329792
Accuracy: 60.22471910112359%
Top 3 accuracy: 98.87640449438202%
Evaluation of Time
Mean Squared Error: 499037382583.2875s	| 5775895.631751012 days
Mean Absolute Error: 419786.0255417727s	| 4.858634554881629 days
Median Absolute Error: 237101.25s	| 2.7442274305555556 days


----------Suffix 4------------
Evaluation of Activity
Log-loss: 0.9269709278757755
Accuracy: 60.0%
Top 3 accuracy: 99.48717948717949%
Evaluation of Time
Mean Squared Error: 244371815325.20825s	| 2828377.4921899103 days
Mean Absolute Error: 225761.01116599058s	| 2.6129746662730393

In [31]:
evalAct(full_gt_a, full_pred_a, targetchartoindice)

Log-loss: 0.8229239157588412
Accuracy: 70.94831911690919%
Top 3 accuracy: 98.7957852483693%


In [32]:
evalTime(full_gt_t, full_pred_t, divisor)

Mean Squared Error: 383403343918.9514s	| 4437538.702765642 days
Mean Absolute Error: 313023.079883964s	| 3.622952313471805 days
Median Absolute Error: 49326.25s	| 0.5709056712962963 days


In [33]:
len(full_gt_a)

1993

# Test single case

## Predict

In [ ]:
sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 =  getSuffix(2, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)

In [ ]:
len(sentences_2), len(next_chars_2)

In [ ]:
X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)

In [ ]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [ ]:
pred[0].shape, pred[1].shape

## Evaluate

### Activity

In [ ]:
evalAct(next_chars_2, pred[0], targetchartoindice)

### Time

In [ ]:
evalTime(next_chars_t_2, pred_t, divisor)